In [2]:
# --- INSTALL & AUTH ---
!pip -q install google-cloud-bigquery pandas-gbq plotly==5.*

from google.colab import auth
auth.authenticate_user()

import pandas as pd
import plotly.express as px
from pandas_gbq import read_gbq

PROJECT_ID = "mgmt-46700"  # 👈 Replace with your GCP project ID
pd.options.display.float_format = "{:,.2f}".format

Prompt:
You are my SQL co-pilot for the public dataset `bigquery-public-data.thelook_ecommerce`.
First, list the tables and briefly summarize their roles and key join keys.
Return a compact diagram (text bullets) showing how orders, order_items, products, users, events, inventory_items relate.


**Tables in `bigquery-public-data.thelook_ecommerce` Dataset:**

*   **orders**: Contains information about customer orders.
    *   Key Join Key: `order_id` (joins with `order_items`)
*   **order_items**: Contains information about individual items within orders.
    *   Key Join Keys: `order_id` (joins with `orders`), `product_id` (joins with `products`), `inventory_item_id` (joins with `inventory_items`)
*   **products**: Contains information about the products sold.
    *   Key Join Key: `id` (joins with `order_items` as `product_id`)
*   **users**: Contains information about the customers.
    *   Key Join Key: `id` (joins with `orders` as `user_id`)
*   **events**: Contains information about user events (e.g., visits, purchases).
    *   Key Join Key: `user_id` (joins with `users`)
*   **inventory_items**: Contains information about the items in inventory.
    *   Key Join Key: `id` (joins with `order_items` as `inventory_item_id`)
*    **distribution_centers**: Contains information about distribution centers.
     *   Key Join Key: `id` (joins with `inventory_items` as `distribution_center_id`)
*   **thelook_ecommerce-table**: This appears to be a view or a combined table, its specific role and join keys would depend on its definition, but it likely aggregates data from other tables.

**Compact Diagram of Relationships:**

*   `users` (id) <--> `orders` (user_id)
*   `orders` (order_id) <--> `order_items` (order_id)
*   `products` (id) <--> `order_items` (product_id)
*   `inventory_items` (id) <--> `order_items` (inventory_item_id)
*   `inventory_items` (distribution_center_id) <--> `distribution_centers` (id)
*   `users` (id) <--> `events` (user_id)

In [ ]:
# 1.1 List tables quickly (optional helper)
from google.cloud import bigquery

# Initialize client
client = bigquery.Client(project=PROJECT_ID)

for t in client.list_tables("bigquery-public-data.thelook_ecommerce"):
    print("-", t.table_id)

- distribution_centers
- events
- inventory_items
- order_items
- orders
- products
- thelook_ecommerce-table
- users


## 1) Discover (AI-Assisted + CTE/Window)
Gemini Prompt (paste into Colab Gemini)

You are my SQL co-pilot for bigquery-public-data.thelook_ecommerce.

Propose 5 growth KPIs suitable for an online retailer (definitions + formulas using available columns).

Write BigQuery SQL with CTEs + window functions to compute monthly Revenue, AOV, Repeat Purchase Rate, plus MoM/YoY for Revenue.

Suggest a 90-day rolling revenue query.

In [21]:
sql_kpis = """
WITH tx AS (
  SELECT
    o.order_id,
    o.user_id,
    DATE(o.created_at) AS order_date,
    oi.sale_price,
    p.cost
  FROM `bigquery-public-data.thelook_ecommerce.order_items` oi
  JOIN `bigquery-public-data.thelook_ecommerce.orders` o USING(order_id)
  JOIN `bigquery-public-data.thelook_ecommerce.products` p ON oi.product_id = p.id
  WHERE o.status = 'Complete'
),
per_user_month AS (
  SELECT
    DATE_TRUNC(order_date, MONTH) AS month,
    user_id,
    COUNT(DISTINCT order_id) AS orders_u,
    SUM(sale_price) AS rev_u
  FROM tx
  GROUP BY 1,2
),
mo AS (
  SELECT
    month,
    SUM(rev_u) AS revenue,
    SUM(rev_u) - SUM(0) AS profit,    -- (optional: replace with SUM(sale_price-cost) if needed)
    SUM(orders_u) AS orders,
    COUNT(DISTINCT user_id) AS buyers,
    SAFE_DIVIDE(COUNTIF(orders_u >= 2), COUNT(DISTINCT user_id)) AS repeat_rate
  FROM per_user_month
  GROUP BY 1
)
SELECT
  month,
  revenue,
  SAFE_DIVIDE(revenue, orders) AS aov,
  repeat_rate,
  SAFE_DIVIDE(revenue - LAG(revenue) OVER(ORDER BY month), LAG(revenue) OVER(ORDER BY month)) AS mom_growth,
  SAFE_DIVIDE(revenue - LAG(revenue, 12) OVER(ORDER BY month), LAG(revenue, 12) OVER(ORDER BY month)) AS yoy_growth
FROM mo
ORDER BY month;
"""
kpis = read_gbq(sql_kpis, project_id=PROJECT_ID, use_bqstorage_api=True)
kpis.tail(8)

Downloading: 100%|██████████|


,month,revenue,aov,repeat_rate,mom_growth,yoy_growth
74,2025-03-01,"78,346.05",86.57,0.01,0.19,0.66
75,2025-04-01,"88,326.79",86.51,0.01,0.13,0.88
76,2025-05-01,"88,266.95",83.27,0.01,-0.00,0.89
77,2025-06-01,"94,688.70",83.94,0.02,0.07,0.93
78,2025-07-01,"113,730.16",86.03,0.01,0.20,0.98
79,2025-08-01,"121,483.53",87.46,0.01,0.07,1.41
80,2025-09-01,"139,721.65",88.77,0.02,0.15,1.42
81,2025-10-01,"173,090.29",87.55,0.07,0.24,1.71


This query calculates several key monthly performance indicators for the business, including revenue, average order value (AOV), repeat purchase rate, and month-over-month (MoM) and year-over-year (YoY) revenue growth. The output DataFrame displays these metrics for each month, allowing you to track the business's performance and growth trends over time

In [22]:
sql_roll = """
WITH daily AS (
  SELECT DATE(o.created_at) AS d, SUM(oi.sale_price) AS revenue
  FROM `bigquery-public-data.thelook_ecommerce.order_items` oi
  JOIN `bigquery-public-data.thelook_ecommerce.orders` o USING(order_id)
  WHERE o.status='Complete'
  GROUP BY d
)
SELECT
  d,
  revenue,
  SUM(revenue) OVER(ORDER BY d ROWS BETWEEN 89 PRECEDING AND CURRENT ROW) AS revenue_90d
FROM daily
ORDER BY d;
"""
roll90 = read_gbq(sql_roll, project_id=PROJECT_ID, use_bqstorage_api=True)
roll90.tail(8)


Downloading: 100%|██████████|


,d,revenue,revenue_90d
2297,2025-10-11,"7,897.00","397,739.22"
2298,2025-10-12,"7,414.80","401,674.05"
2299,2025-10-13,"11,152.30","408,133.43"
2300,2025-10-14,"13,681.57","419,950.54"
2301,2025-10-15,"18,006.00","433,427.42"
2302,2025-10-16,"33,579.91","462,750.96"
2303,2025-10-17,"9,667.63","467,613.29"
2304,2025-10-18,"12,489.55","474,331.12"


This query calculates the daily revenue and then computes the rolling sum of revenue over the preceding 90 days for each day. The output shows the date (d), the revenue for that specific day (revenue), and the total revenue accumulated over the last 90 days ending on that date (revenue_90d). This helps to smooth out daily fluctuations and reveal the underlying trend in revenue over a longer period.

## 2) Investigate (one product category + one customer segment)



Prompt:

You are my SQL copilot for the dataset bigquery-public-data.thelook_ecommerce.
Write a BigQuery SQL query to compare product categories by total revenue, total profit, and average sale price.
Use order_items, products, and orders tables, include only status = 'Complete', and ensure results are not empty.
Return rounded values and order by total revenue descending.

In [4]:
# Install & authenticate (run once)
!pip -q install google-cloud-bigquery pandas-gbq
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
import pandas as pd

# Initialize client
client = bigquery.Client(project=PROJECT_ID)

# SQL query to analyze category performance
sql_category_performance = """
SELECT
  p.category,
  ROUND(SUM(oi.sale_price), 2) AS total_revenue,
  ROUND(SUM(oi.sale_price - p.cost), 2) AS total_profit,
  ROUND(AVG(oi.sale_price), 2) AS avg_sale_price,
  COUNT(oi.order_id) AS items_sold
FROM `bigquery-public-data.thelook_ecommerce.order_items` AS oi
JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
  ON oi.product_id = p.id
JOIN `bigquery-public-data.thelook_ecommerce.orders` AS o
  ON oi.order_id = o.order_id
WHERE o.status = 'Complete'
GROUP BY p.category
ORDER BY total_revenue DESC
"""

# Run query and save results in DataFrame
df_category_perf = client.query(sql_category_performance).to_dataframe()

print("✅ Total Revenue, Profit, and Avg Sale Price by Product Category:\n")
df_category_perf.head(10)

✅ Total Revenue, Profit, and Avg Sale Price by Product Category:



,category,total_revenue,total_profit,avg_sale_price,items_sold
0,Outerwear & Coats,"326,380.45","181,412.88",142.77,2286
1,Jeans,"306,763.87","142,252.01",97.45,3148
2,Sweaters,"200,962.85","104,235.65",74.40,2701
3,Suits & Sport Coats,"161,362.06","96,531.96",128.58,1255
4,Fashion Hoodies & Sweatshirts,"160,954.27","76,926.25",55.62,2894
5,Swim,"156,111.63","76,906.92",57.02,2738
6,Sleep & Lounge,"140,348.90","72,850.09",50.00,2807
7,Shorts,"132,864.76","66,370.61",46.78,2840
8,Tops & Tees,"125,112.11","54,921.33",41.47,3017
9,Intimates,"115,084.80","53,856.16",34.24,3361


The top revenue category is Outerwear & Coats. It has an average sale price of roughly 142.77.

Profit margins are strongest in Outerwear & Coats (calculated as total_profit / total_revenue, 181412.88 / 326380.45 = 0.555). This indicates potential for premium positioning.

This helps us select a category for deeper brand and customer investigation.

Prompt:

Based on the top category from the table above, write a BigQuery SQL query to find the top 10 brands ranked by total revenue.
Then, write a second query to identify the top 5 customer regions for that same category, showing total revenue and average profit margin.

In [5]:
# Get the top category from the previously generated DataFrame
# Assuming df_category_perf is available from the previous step
top_category = df_category_perf.iloc[0]['category']
print(f"Analyzing the top category: {top_category}")

# SQL query to find the top 10 brands in the top category
sql_top_brands = f"""
SELECT
  p.brand,
  ROUND(SUM(oi.sale_price), 2) AS total_revenue,
  COUNT(oi.order_id) AS items_sold
FROM `bigquery-public-data.thelook_ecommerce.order_items` AS oi
JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
  ON oi.product_id = p.id
JOIN `bigquery-public-data.thelook_ecommerce.orders` AS o
  ON oi.order_id = o.order_id
WHERE o.status = 'Complete'
  AND p.category = '{top_category}'
GROUP BY p.brand
ORDER BY total_revenue DESC
LIMIT 10
"""

# Run query and save results in DataFrame
df_top_brands = client.query(sql_top_brands).to_dataframe()

print(f"\n✅ Top 10 Brands in '{top_category}' by Revenue:\n")
display(df_top_brands)

Analyzing the top category: Outerwear & Coats

✅ Top 10 Brands in 'Outerwear & Coats' by Revenue:



,brand,total_revenue,items_sold
0,Carhartt,"22,315.16",177
1,Arc'teryx,"14,894.80",46
2,The North Face,"14,401.68",31
3,Columbia,"12,233.59",128
4,Mountain Hardwear,"11,879.73",58
5,Calvin Klein,"11,112.03",80
6,Canada Goose,"10,644.95",19
7,Alpha Industries,"8,230.72",36
8,Barbour,"7,972.97",28
9,Kenneth Cole,"7,427.56",64


In [6]:
# Get the top category from the previously generated DataFrame
# Assuming df_category_perf is available from the previous step
top_category = df_category_perf.iloc[0]['category']
print(f"Analyzing the top category: {top_category}")

# SQL query to identify top 5 customer regions for the top category
sql_top_regions = f"""
SELECT
  u.country AS region,
  ROUND(SUM(oi.sale_price), 2) AS total_revenue,
  ROUND(AVG(oi.sale_price - p.cost), 2) AS avg_profit_margin
FROM `bigquery-public-data.thelook_ecommerce.order_items` AS oi
JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
  ON oi.product_id = p.id
JOIN `bigquery-public-data.thelook_ecommerce.orders` AS o
  ON oi.order_id = o.order_id
JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
  ON o.user_id = u.id
WHERE o.status = 'Complete'
  AND p.category = '{top_category}'
GROUP BY region
ORDER BY total_revenue DESC
LIMIT 5
"""

# Run query and save results in DataFrame
df_top_regions = client.query(sql_top_regions).to_dataframe()

print(f"\n✅ Top 5 Customer Regions for '{top_category}' by Revenue:\n")
display(df_top_regions)

Analyzing the top category: Outerwear & Coats

✅ Top 5 Customer Regions for 'Outerwear & Coats' by Revenue:



,region,total_revenue,avg_profit_margin
0,China,"115,923.75",83.91
1,United States,"72,127.61",78.40
2,Brasil,"46,798.99",72.08
3,United Kingdom,"16,008.99",79.04
4,South Korea,"15,674.31",74.69


The strongest margins for the 'Outerwear & Coats' category appear to come from China and the United Kingdom, suggesting potential for premium pricing in these regions. (Note: The average profit margin calculation in the region query is an average of the individual item profit margins, not the overall margin percentage for the region's total revenue. A separate calculation would be needed for the overall margin percentage)

## Validate insights

### Subtask:
Cross-check the top category revenue and profit insights by running a simplified query focusing only on these metrics per category.


**Reasoning**:
Write and execute a BigQuery SQL query to calculate total revenue and total profit by product category, filtered by complete orders, and display the top results to validate previous findings.



In [7]:
# SQL query to cross-check category revenue and profit
sql_category_validation = """
SELECT
  p.category,
  ROUND(SUM(oi.sale_price), 2) AS total_revenue,
  ROUND(SUM(oi.sale_price - p.cost), 2) AS total_profit
FROM `bigquery-public-data.thelook_ecommerce.order_items` AS oi
JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
  ON oi.product_id = p.id
JOIN `bigquery-public-data.thelook_ecommerce.orders` AS o
  ON oi.order_id = o.order_id
WHERE o.status = 'Complete'
GROUP BY p.category
ORDER BY total_revenue DESC
"""

# Run query and save results in DataFrame
df_category_validation = client.query(sql_category_validation).to_dataframe()

print("✅ Cross-checking Total Revenue and Profit by Product Category:\n")
display(df_category_validation.head(10))

✅ Cross-checking Total Revenue and Profit by Product Category:



,category,total_revenue,total_profit
0,Outerwear & Coats,"326,380.45","181,412.88"
1,Jeans,"306,763.87","142,252.01"
2,Sweaters,"200,962.85","104,235.65"
3,Suits & Sport Coats,"161,362.06","96,531.96"
4,Fashion Hoodies & Sweatshirts,"160,954.27","76,926.25"
5,Swim,"156,111.63","76,906.92"
6,Sleep & Lounge,"140,348.90","72,850.09"
7,Shorts,"132,864.76","66,370.61"
8,Tops & Tees,"125,112.11","54,921.33"
9,Intimates,"115,084.80","53,856.16"


## Validate insights

### Subtask:
Cross-check the top region profit margin insight for the top category by running a query that calculates the overall profit margin percentage for each region instead of the average of individual item profit margins.


**Reasoning**:
Get the top category, construct and execute the SQL query to calculate overall profit margin percentage by region for that category, and display the result.



In [8]:
# Get the top category from the previously generated DataFrame
# Assuming df_category_perf is available from the previous step
top_category = df_category_perf.iloc[0]['category']
print(f"Analyzing the top category: {top_category}")

# SQL query to identify top 5 customer regions for the top category with overall profit margin percentage
sql_top_regions_profit_margin = f"""
SELECT
  u.country AS region,
  ROUND(SUM(oi.sale_price), 2) AS total_revenue,
  ROUND(SUM(oi.sale_price - p.cost), 2) AS total_profit,
  ROUND(SAFE_DIVIDE(SUM(oi.sale_price - p.cost), SUM(oi.sale_price)) * 100, 2) AS overall_profit_margin_pct
FROM `bigquery-public-data.thelook_ecommerce.order_items` AS oi
JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
  ON oi.product_id = p.id
JOIN `bigquery-public-data.thelook_ecommerce.orders` AS o
  ON oi.order_id = o.order_id
JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
  ON o.user_id = u.id
WHERE o.status = 'Complete'
  AND p.category = '{top_category}'
GROUP BY region
ORDER BY total_revenue DESC
LIMIT 5
"""

# Run query and save results in DataFrame
df_top_regions_profit_margin = client.query(sql_top_regions_profit_margin).to_dataframe()

print(f"\n✅ Top 5 Customer Regions for '{top_category}' by Revenue with Overall Profit Margin Percentage:\n")
display(df_top_regions_profit_margin)

Analyzing the top category: Outerwear & Coats

✅ Top 5 Customer Regions for 'Outerwear & Coats' by Revenue with Overall Profit Margin Percentage:



,region,total_revenue,total_profit,overall_profit_margin_pct
0,China,"115,923.75","64,355.87",55.52
1,United States,"72,127.61","40,139.05",55.65
2,Brasil,"46,798.99","25,877.07",55.29
3,United Kingdom,"16,008.99","8,931.09",55.79
4,South Korea,"15,674.31","8,739.25",55.76


## 3) Validate — Cross-checks & Counterexamples
Subtask

Cross-check at least two AI insights and show one case where the first answer was misleading and how you corrected it.

Reasoning

We’ll re-compute results from different angles:

Brand leader — is leadership driven by AOV or volume?

Regions — does highest revenue also mean best margin?

Misleading case — do deep discounts really “drive performance,” or do they erode margins?

In [18]:
# SQL query to analyze brand performance by revenue, AOV, and items sold for the top category
sql_brand_performance_validation = f"""
SELECT
  p.brand,
  ROUND(SUM(oi.sale_price), 2) AS total_revenue,
  ROUND(AVG(oi.sale_price), 2) AS average_order_value,
  COUNT(oi.order_id) AS items_sold
FROM `bigquery-public-data.thelook_ecommerce.order_items` AS oi
JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
  ON oi.product_id = p.id
JOIN `bigquery-public-data.thelook_ecommerce.orders` AS o
  ON oi.order_id = o.order_id
WHERE o.status = 'Complete'
  AND p.category = '{top_category}' -- Use the top category identified earlier
GROUP BY p.brand
ORDER BY total_revenue DESC
LIMIT 10 -- Limiting to top brands by revenue for comparison
"""

# Run query and save results in DataFrame
df_brand_performance_validation = client.query(sql_brand_performance_validation).to_dataframe()

print(f"✅ Brand Performance in '{top_category}' (Revenue, AOV, Items Sold):\n")
display(df_brand_performance_validation)

# Analyze brand leadership drivers
print("\nAnalysis of Brand Leadership Drivers:")
print(f"Top brand by revenue: {df_brand_performance_validation.iloc[0]['brand']}")
print(f"Top brand by items sold: {df_brand_performance_validation.sort_values(by='items_sold', ascending=False).iloc[0]['brand']}")
print(f"Top brand by AOV: {df_brand_performance_validation.sort_values(by='average_order_value', ascending=False).iloc[0]['brand']}")

✅ Brand Performance in 'Outerwear & Coats' (Revenue, AOV, Items Sold):



,brand,total_revenue,average_order_value,items_sold
0,Carhartt,"22,315.16",126.07,177
1,Arc'teryx,"14,894.80",323.80,46
2,The North Face,"14,401.68",464.57,31
3,Columbia,"12,233.59",95.57,128
4,Mountain Hardwear,"11,879.73",204.82,58
5,Calvin Klein,"11,112.03",138.90,80
6,Canada Goose,"10,644.95",560.26,19
7,Alpha Industries,"8,230.72",228.63,36
8,Barbour,"7,972.97",284.75,28
9,Kenneth Cole,"7,427.56",116.06,64



Analysis of Brand Leadership Drivers:
Top brand by revenue: Carhartt
Top brand by items sold: Carhartt
Top brand by AOV: Canada Goose


In [19]:
# SQL query to analyze regional performance by revenue and overall profit margin percentage for the top category
sql_region_margin_validation = f"""
SELECT
  u.country AS region,
  ROUND(SUM(oi.sale_price), 2) AS total_revenue,
  ROUND(SUM(oi.sale_price - p.cost), 2) AS total_profit,
  ROUND(SAFE_DIVIDE(SUM(oi.sale_price - p.cost), SUM(oi.sale_price)) * 100, 2) AS overall_profit_margin_pct
FROM `bigquery-public-data.thelook_ecommerce.order_items` AS oi
JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
  ON oi.product_id = p.id
JOIN `bigquery-public-data.thelook_ecommerce.orders` AS o
  ON oi.order_id = o.order_id
JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
  ON o.user_id = u.id
WHERE o.status = 'Complete'
  AND p.category = '{top_category}' -- Use the top category identified earlier
GROUP BY region
HAVING total_revenue > 0 -- Exclude regions with no revenue
ORDER BY total_revenue DESC
LIMIT 10 -- Limiting to top regions by revenue for comparison
"""

# Run query and save results in DataFrame
df_region_margin_validation = client.query(sql_region_margin_validation).to_dataframe()

print(f"✅ Regional Performance in '{top_category}' (Revenue and Overall Profit Margin %):\n")
display(df_region_margin_validation)

# Analyze regional performance drivers
print("\nAnalysis of Regional Performance:")
print(f"Top region by revenue: {df_region_margin_validation.iloc[0]['region']}")
print(f"Top region by overall profit margin %: {df_region_margin_validation.sort_values(by='overall_profit_margin_pct', ascending=False).iloc[0]['region']}")

✅ Regional Performance in 'Outerwear & Coats' (Revenue and Overall Profit Margin %):



,region,total_revenue,total_profit,overall_profit_margin_pct
0,China,"115,923.75","64,355.87",55.52
1,United States,"72,127.61","40,139.05",55.65
2,Brasil,"46,798.99","25,877.07",55.29
3,United Kingdom,"16,008.99","8,931.09",55.79
4,South Korea,"15,674.31","8,739.25",55.76
5,Germany,"14,409.08","7,987.63",55.43
6,France,"12,377.42","6,974.82",56.35
7,Spain,"12,193.94","6,769.86",55.52
8,Japan,"7,808.90","4,354.27",55.76
9,Australia,"7,642.31","4,281.19",56.02



Analysis of Regional Performance:
Top region by revenue: China
Top region by overall profit margin %: France


In [26]:
# SQL query to analyze the impact of discounts on revenue and profit margin for the top category
sql_discount_impact = f"""
SELECT
  CASE
    WHEN SAFE_DIVIDE(p.retail_price - oi.sale_price, p.retail_price) IS NULL THEN 'Unknown'
    WHEN SAFE_DIVIDE(p.retail_price - oi.sale_price, p.retail_price) < 0.05 THEN '0–5%'
    WHEN SAFE_DIVIDE(p.retail_price - oi.sale_price, p.retail_price) < 0.15 THEN '5–15%'
    WHEN SAFE_DIVIDE(p.retail_price - oi.sale_price, p.retail_price) < 0.30 THEN '15–30%'
    WHEN SAFE_DIVIDE(p.retail_price - oi.sale_price, p.retail_price) < 0.50 THEN '30–50%'
    ELSE '50%+'
  END AS discount_bucket,
  COUNT(oi.order_id) AS items_sold,
  ROUND(SUM(oi.sale_price), 2) AS total_revenue,
  ROUND(SUM(oi.sale_price - p.cost), 2) AS total_profit,
  ROUND(SAFE_DIVIDE(SUM(oi.sale_price - p.cost), SUM(oi.sale_price)) * 100, 2) AS overall_profit_margin_pct
FROM `bigquery-public-data.thelook_ecommerce.order_items` AS oi
JOIN `bigquery-public-data.thelook_ecommerce.orders` AS o
  ON oi.order_id = o.order_id
JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
  ON oi.product_id = p.id
WHERE o.status = 'Complete'
  AND p.category = '{top_category}' -- Use the top category identified earlier
GROUP BY discount_bucket
ORDER BY total_revenue DESC
"""

# Run query and save results in DataFrame
df_discount_impact = client.query(sql_discount_impact).to_dataframe()

print(f"✅ Impact of Discounts on Revenue and Profit Margin in '{top_category}':\n")
display(df_discount_impact)



✅ Impact of Discounts on Revenue and Profit Margin in 'Outerwear & Coats':



,discount_bucket,items_sold,total_revenue,total_profit,overall_profit_margin_pct
0,0–5%,2286,"326,380.45","181,412.88",55.58


In [23]:
print("Comparison of Profit Insights for Top Categories:")
display(df_category_perf[['category', 'total_revenue', 'total_profit']].head())
display(df_category_validation.head())

print("\nComparison of Profit Margin Insights for Top Regions in Top Category:")
display(df_top_regions.head())
display(df_top_regions_profit_margin.head())

print("\nAnalysis of Discrepancies and Refined Interpretations:")

# 1. Compare total profit for top categories
print("\nComparing Total Profit from df_category_perf and df_category_validation:")
merged_category_profit = pd.merge(df_category_perf[['category', 'total_profit']],
                                   df_category_validation,
                                   on='category',
                                   suffixes=('_perf', '_validation'))
merged_category_profit['profit_difference'] = merged_category_profit['total_profit_perf'] - merged_category_profit['total_profit_validation']
print("Difference in Total Profit Calculation (df_category_perf - df_category_validation):")
display(merged_category_profit.head())

# Discrepancy analysis for category profit
if merged_category_profit['profit_difference'].abs().sum() < 0.01:
    print("No significant discrepancies found in total profit calculation for top categories between the two dataframes.")
    print("The initial interpretation about profit margins based on total profit and total revenue from df_category_perf is validated.")
else:
    print("Discrepancies found in total profit calculation for top categories.")
    print("The cause is likely due to rounding differences or potential variations in the exact data pulled, although the column names are the same.")
    print("For interpreting overall profitability of categories, 'total_profit' provides the absolute profit figure, while the ratio of 'total_profit' to 'total_revenue' gives the overall profit margin percentage.")
    print("The initial interpretation of Outerwear & Coats having strong margins (0.555 ratio) based on df_category_perf is likely still valid despite minor rounding differences.")


# 2. Compare profit margin insights for top regions
print("\nComparing Profit Margin Insights from df_top_regions and df_top_regions_profit_margin:")

print("\ndf_top_regions (Average Item Profit Margin):")
display(df_top_regions.head())

print("\ndf_top_regions_profit_margin (Overall Profit Margin Percentage):")
display(df_top_regions_profit_margin.head())

# Discrepancy analysis for regional profit margin
print("\nDiscussion on Regional Profit Margin Calculations:")
print("df_top_regions shows 'avg_profit_margin', which is the average of the (sale_price - cost) for each item sold in that region and category.")
print("df_top_regions_profit_margin shows 'overall_profit_margin_pct', which is calculated as (SUM(sale_price - cost) / SUM(sale_price)) * 100 for all items in that region and category.")
print("\nThe 'avg_profit_margin' tells us the average absolute profit per item, while the 'overall_profit_margin_pct' tells us the profit as a percentage of revenue.")
print("The initial interpretation noted that 'average profit margin calculation in the region query is an average of the individual item profit margins, not the overall margin percentage'. This was correct.")
print("For understanding which regions are most profitable relative to the revenue they generate, the 'overall_profit_margin_pct' is a more appropriate metric.")

# Corrected interpretation for regional profit margins
print("\nRefined Interpretation of Regional Profit Margins:")
print("Based on 'overall_profit_margin_pct' from df_top_regions_profit_margin, the top regions for 'Outerwear & Coats' have very similar overall profit margins, all hovering around 55-56%.")
print("While China and the United Kingdom had slightly higher 'avg_profit_margin' (absolute profit per item) in df_top_regions, their 'overall_profit_margin_pct' is comparable to other top regions.")
print("Therefore, the conclusion about 'strongest margins' from China and the United Kingdom from the previous interpretation should be refined to note that while they might have slightly higher absolute profit per item, their overall profitability as a percentage of revenue is similar to other top regions for this category.")

Comparison of Profit Insights for Top Categories:


,category,total_revenue,total_profit
0,Outerwear & Coats,"326,380.45","181,412.88"
1,Jeans,"306,763.87","142,252.01"
2,Sweaters,"200,962.85","104,235.65"
3,Suits & Sport Coats,"161,362.06","96,531.96"
4,Fashion Hoodies & Sweatshirts,"160,954.27","76,926.25"


,category,total_revenue,total_profit
0,Outerwear & Coats,"326,380.45","181,412.88"
1,Jeans,"306,763.87","142,252.01"
2,Sweaters,"200,962.85","104,235.65"
3,Suits & Sport Coats,"161,362.06","96,531.96"
4,Fashion Hoodies & Sweatshirts,"160,954.27","76,926.25"



Comparison of Profit Margin Insights for Top Regions in Top Category:


,region,total_revenue,avg_profit_margin
0,China,"115,923.75",83.91
1,United States,"72,127.61",78.40
2,Brasil,"46,798.99",72.08
3,United Kingdom,"16,008.99",79.04
4,South Korea,"15,674.31",74.69


...

No significant discrepancies found in total profit calculation for top categories between the two dataframes.
The initial interpretation about profit margins based on total profit and total revenue from df_category_perf is validated.

Comparing Profit Margin Insights from df_top_regions and df_top_regions_profit_margin:

df_top_regions (Average Item Profit Margin):


,region,total_revenue,avg_profit_margin
0,China,"115,923.75",83.91
1,United States,"72,127.61",78.40
2,Brasil,"46,798.99",72.08
3,United Kingdom,"16,008.99",79.04
4,South Korea,"15,674.31",74.69



df_top_regions_profit_margin (Overall Profit Margin Percentage):


,region,total_revenue,total_profit,overall_profit_margin_pct
0,China,"115,923.75","64,355.87",55.52
1,United States,"72,127.61","40,139.05",55.65
2,Brasil,"46,798.99","25,877.07",55.29
3,United Kingdom,"16,008.99","8,931.09",55.79
4,South Korea,"15,674.31","8,739.25",55.76



Discussion on Regional Profit Margin Calculations:
df_top_regions shows 'avg_profit_margin', which is the average of the (sale_price - cost) for each item sold in that region and category.
df_top_regions_profit_margin shows 'overall_profit_margin_pct', which is calculated as (SUM(sale_price - cost) / SUM(sale_price)) * 100 for all items in that region and category.

The 'avg_profit_margin' tells us the average absolute profit per item, while the 'overall_profit_margin_pct' tells us the profit as a percentage of revenue.
The initial interpretation noted that 'average profit margin calculation in the region query is an average of the individual item profit margins, not the overall margin percentage'. This was correct.
For understanding which regions are most profitable relative to the revenue they generate, the 'overall_profit_margin_pct' is a more appropriate metric.

Refined Interpretation of Regional Profit Margins:
Based on 'overall_profit_margin_pct' from df_top_regions_profit_marg

Limited Discounting in this Category: The retailer might not heavily discount 'Outerwear & Coats'.

Data Anomaly: There might be an issue with how discounts are recorded or calculated in the data for this specific category or for completed orders.
Misleading Insight: If there were significant discounts, we would expect to see revenue and profit distributed across different discount buckets. The current result makes it difficult to analyze the impact of deeper discounts on performance and margins for this category, as there's no data in the higher discount buckets to compare against.

Therefore, based on this specific data for 'Outerwear & Coats' and completed orders, we cannot definitively conclude whether deep discounts drive performance or erode margins because we lack data points in the higher discount tiers. A broader analysis across all categories or an investigation into the data recording for discounts might be necessary to draw more general conclusions.

## Extend (Communication):

Build one interactive Plotly chart in Colab and one executive Looker Studio dashboard with:

Scorecard: revenue (or profit), last 30 days

Pie/Donut: sales % by region or channel

Bar: top 5 products/categories

Write 1–2 specific recommendations using the Strategist pattern.

In [28]:
# Define the number of top categories to display
N = 5

# Filter the DataFrame to include only the top N categories
df_top_n_categories = df_category_perf.head(N).copy()

# Create a grouped bar chart using Plotly Express
fig = px.bar(df_top_n_categories,
             x='category',
             y=['total_revenue', 'total_profit'],
             title=f'Top {N} Category Revenue and Profit',
             labels={'value': 'Amount', 'variable': 'Metric'},
             barmode='group')

# Update layout for better readability if needed (optional)
fig.update_layout(xaxis_title='Product Category',
                  yaxis_title='Amount ($)',
                  legend_title='Metric')

# Display the chart
fig.show()

In [29]:
# Assuming df_top_brands is available from previous steps

# Create a bar chart for top 10 brands by revenue
fig_brands = px.bar(df_top_brands,
                    x='brand',
                    y='total_revenue',
                    title=f'Top 10 Brands in {top_category} by Revenue',
                    labels={'brand': 'Brand', 'total_revenue': 'Total Revenue ($)'})

# Update layout for better readability (optional)
fig_brands.update_layout(xaxis_title='Brand',
                         yaxis_title='Total Revenue ($)')

# Display the chart
fig_brands.show()

In [30]:
# Assuming df_top_regions is available from previous steps

# Create a bar chart for top 5 regions by revenue
fig_regions = px.bar(df_top_regions,
                     x='region',
                     y='total_revenue',
                     title=f'Top 5 Customer Regions for {top_category} by Revenue',
                     labels={'region': 'Region', 'total_revenue': 'Total Revenue ($)'})

# Update layout for better readability (optional)
fig_regions.update_layout(xaxis_title='Region',
                          yaxis_title='Total Revenue ($)')

# Display the chart
fig_regions.show()

KPIs & Looker Studio Dashboard

Dashboard Link:

https://lookerstudio.google.com/reporting/95bfb9ba-1723-40b6-8347-14e2669e381d

## Strategic Recommendations (Strategist Pattern)

Based on the analysis of category, brand, and regional performance for 'Outerwear & Coats', here are a couple of strategic recommendations:

### Recommendation 1: Capitalize on High-Margin Regions

**Strategist Pattern**: *Identify a key insight and propose an action to leverage it for strategic advantage.*

**Insight**: The 'Outerwear & Coats' category has strong overall profit margins, and regions like China and the United Kingdom show slightly higher average profit per item, with overall profit margins comparable to other top regions (around 55-56%).

**Recommendation**: Develop targeted marketing campaigns and potentially localized product offerings specifically for the China and United Kingdom markets to capitalize on the strong profitability in these regions for the 'Outerwear & Coats' category. This could involve highlighting premium aspects of the products or tailoring messaging to resonate with customers in these areas.

### Recommendation 2: Analyze Brand Performance Beyond Revenue

**Strategist Pattern**: *Analyze contributing factors to performance and suggest actions to optimize based on a deeper understanding.*

**Insight**: While Carhartt is the top brand in 'Outerwear & Coats' by both revenue and items sold, brands like Canada Goose and The North Face have significantly higher average order values (AOV).

**Recommendation**: Investigate the customer base and marketing strategies of high-AOV brands like Canada Goose and The North Face within the 'Outerwear & Coats' category. Understand what drives their higher average transaction value. Apply these learnings, where applicable and aligned with brand positioning, to other brands in the category or explore opportunities to introduce more premium or higher-priced items within the category to potentially increase overall AOV.